In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
#There are deleted users among these votes. To Take into account only active users refer to  ExploratoryPlotsByGroups.ipynb
votes = pd.DataFrame.from_csv('c:/Users/Joel/Predictive-model-for-the-voting-system-in-SE-network/spanish_site/AnsVotes_TSeries.csv', index_col=None)
votes.head()

,QuestionId,AnsId,Age_vot,AnsWordCount,AcceptedAnsId,QScore,QVotes,Score,Votes,Upvotes,...,CumScore,ReScore,QReScore,Comments,AnsRank,ReScore_rank,Ans_count,Pbias,DRank,EPbias
0,1,56,0,201,56,0,0,2,2,2,...,0,0,0,0,1,1,1,1,0,1
1,2,10,0,145,10,0,0,3,3,3,...,0,0,0,0,1,1,1,1,0,1
2,2,10,0,145,10,0,0,3,3,3,...,0,0,0,0,1,1,1,1,0,1
3,3,9,0,25,109,0,0,4,4,4,...,0,0,0,0,1,1,1,1,0,1
4,7,16,0,45,2977,0,0,3,3,3,...,0,0,0,0,1,1,1,1,0,1


In [13]:
votes['Norm_Pos'] = votes['AnsRank']/votes['Ans_count']
votes['Norm_DRank'] = votes['DRank']/votes['Ans_count']

In [14]:
votes.shape

(23328, 24)

### Distribution of : number of days an answer is voted on (at least one vote)

In [15]:
votes.groupby('AnsId').count().Age_vot.value_counts().head(15)

1     1638
2     1405
3      740
4      598
6      292
5      217
8      153
12      97
10      71
9       62
7       45
16      31
18      28
15      25
24      25
Name: Age_vot, dtype: int64

# Regression happens on votes the day after attributes were measured

In [16]:
select = lambda df,one_vote=True: df[['Age_vot','Norm_Pos','Norm_DRank','ReScore','Votes']]if one_vote else df[['Age_vot','Norm_Pos','Norm_DRank','ReScore']]
shift_select = lambda df: select(df) if df.shape[0]==1 else pd.concat([select(df,one_vote=False),df['Votes'].shift(-1)],axis=1)
sort_shift = lambda df: shift_select(df.sort_values(by='Age_vot'))
def shift_votes(df):
    s_df = sort_shift(df)
    s_df.loc[:,'Votes'] = s_df.Votes.fillna(method='ffill')
    return s_df[s_df.Votes.notnull()]

In [17]:
votes = votes.groupby(['AnsId']).apply(shift_votes).reset_index(level=[0],drop=False)

In [18]:
votes.shape #shape should be the same

(23328, 6)

In [20]:
votes.groupby('AnsId').count().Age_vot.value_counts().head(15)

1     1638
2     1405
3      740
4      598
6      292
5      217
8      153
12      97
10      71
9       62
7       45
16      31
18      28
15      25
24      25
Name: Age_vot, dtype: int64

In [22]:
votes.head(20)

,AnsId,Age_vot,Norm_Pos,Norm_DRank,ReScore,Votes
7132,4,0,1.000000,0.000000,0.000000,1
21434,4,1436,0.500000,-0.333333,0.250000,1
13998,5,0,1.000000,0.000000,0.000000,1
20327,5,108,1.000000,0.400000,0.500000,1
20328,6,0,1.000000,0.000000,0.000000,1
19213,8,0,0.750000,0.500000,0.000000,1
3,9,0,1.000000,0.000000,0.000000,1
22248,9,12,0.250000,0.000000,0.800000,1
22249,9,361,0.250000,0.000000,0.833333,1
22250,9,400,0.250000,0.000000,0.857143,1


In [27]:
def get_intervals(att,quantiles):
    qtls = votes[att].quantile(quantiles).values
    return dict(zip(zip(qtls,qtls[1:]),[e for e in range(len(zip(qtls,qtls[1:])))]))
    
def get_group(intervals,pred):
    for l,u in intervals:
        if l<=pred and pred<=u:
            return intervals[(l,u)]
        
def reverse_groups(groups,unit='days'):
    return {groups[(d1,d2)]:"%.2f to %.2f %s"%(d1,d2,unit) for d1,d2 in groups}

In [24]:
from functools import partial

age_intervals = get_intervals('Age_vot',[0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.0])
rep_intervals = get_intervals('Reputation',[0,.25,.5,.75,1.0])
pos_intervals = get_intervals('AnsRank',[0,.25,.5,.75,1.0])
rscore_intervals = get_intervals('ReScore',[0,.25,.5,.75,1.0])
anscount_intervals = get_intervals('Ans_count',[0,.25,.5,.75,1.0])

get_age_group = partial(get_group,age_intervals)
get_rep_group = partial(get_group,rep_intervals)
get_pos_group = partial(get_group,pos_intervals)
get_rscore_group = partial(get_group,rscore_intervals)
get_anscount_group = partial(get_group,anscount_intervals)


In [25]:
votes['Age_group'] = votes['Age_vot'].apply(get_age_group)
votes['Rep_group'] = votes['Reputation'].apply(get_rep_group)
votes['Pos_group'] = votes['AnsRank'].apply(get_age_group)
votes['RScore_group'] = votes['ReScore'].apply(get_age_group)
votes['AnsCount_group'] = votes['Ans_count'].apply(get_age_group)

In [28]:
rev_age_groups = reverse_groups(age_intervals)
rev_rep_groups = reverse_groups(rep_intervals,unit='Points')
rev_pos_groups = reverse_groups(pos_intervals,unit='Position')
rev_rscore_groups = reverse_groups(rscore_intervals,unit='Relative Score')
rev_anscount_groups = reverse_groups(anscount_intervals,unit='Total Answers in Question')

In [32]:
rev_anscount_groups

{0: '1.00 to 1.00 Total Answers in Question',
 1: '1.00 to 2.00 Total Answers in Question',
 2: '2.00 to 3.00 Total Answers in Question',
 3: '3.00 to 12.00 Total Answers in Question'}